Getting data
rows :12477 
columns : 29
header : ['id' 'name' 'type' 'frameType' 'desc' 'race' 'archetype' 'card_sets'
 'card_images' 'card_prices' 'atk' 'def' 'level' 'attribute' 'scale'
 'linkval' 'linkmarkers' 'banlist_info']

In [251]:
import pandas as pd
import requests
import tkinter as tk
from PIL import ImageTk, Image
from io import BytesIO
from datetime import datetime

In [252]:
i = 0
try:
    respFromApi = requests.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")
    respFromApi.raise_for_status()
    resp = respFromApi.json()
    data = pd.DataFrame(resp["data"])
    print(data.columns.values)
    print(data.shape[0], data.shape[1])
except Exception as error:
    print(
        f"Exception : {error.__class__.__name__} -- strerror: {error}")

['id' 'name' 'type' 'frameType' 'desc' 'race' 'archetype' 'card_sets'
 'card_images' 'card_prices' 'atk' 'def' 'level' 'attribute' 'scale'
 'linkval' 'linkmarkers' 'banlist_info']
12516 18


In [253]:
def GetImage():
    try :
        global i
        url = data['card_images'][i][0]['image_url']
        image_response = requests.get(url)
        img_data = image_response.content
        return(ImageTk.PhotoImage(Image.open(BytesIO(img_data))))
    except Exception as error:
        print(
            f"Exception : {error.__class__.__name__} -- strerror: {error}")


In [286]:
def GetPrice():
    global i
    priceCardMarket = 0
    priceCardMarket = data['card_prices'][i][0]['cardmarket_price']
    if priceCardMarket:
        return(priceCardMarket)
    else :
        return(0)
   

In [287]:

def GetFamily():
    global i
    family = []
    if(isinstance(data['card_sets'][i],list)):
        for f in data['card_sets'][i]:
            family.append(f["set_code"])
        return(family) 
    else:
        return "Not Found"
    

old api : https://yugiohprices.docs.apiary.io/#
datetimeStr = jsonPrice["data"][0]["price_data"]["data"]["prices"]["updated_at"][0:19]
datetime_object = datetime.strptime(datetimeStr, '%Y-%m-%d %H:%M:%S')
price = jsonPrice["data"][0]["price_data"]["data"]["prices"]["average"]

In [288]:
def clicked(labelImage:tk.Label, labelPrice:tk.Label, labelFamily:tk.Label):
    global i
    img = GetImage()
    labelImage.configure(image=img)
    labelImage.image=img
    price = GetPrice()
    family = GetFamily()
    labelFamily.configure(text = f"{family}")
    if(price):
        labelPrice.configure(text = f"Price: {price} €")
    else:
        labelPrice.configure(text = "Not found") 
   


In [289]:
def SearchById(entry, labelImage, labelPrice, labelFamily):
    getInput = entry.get()
    global i
    try:
        findInput = data[data["id"] == int(getInput)]
        i = findInput.index[0]
        clicked(labelImage, labelPrice, labelFamily)
    except Exception as error:
        print(f"Exception : {error.__class__.__name__} -- strerror: {error}")
    
    

In [290]:
def Click(*args):
    global i
    clicked(args[0], args[1],args[2])
    if(args[3]):
        if(i < data.shape[0]):
            i+=1
    else:
        if(i > 0) :
            i-=1

In [291]:
class GUI():
    def __init__(self) -> None:
        self.window = tk.Tk()
        self.inputId = tk.Entry()
        self.inputId.insert(0,"CardId: ex 02656842")
        self.labelImage = tk.Label(master=self.window, image = None)
        self.labelPrice = tk.Label(master=self.window, text = None)
        self.labelFamily = tk.Label(master=self.window,text = None)
        self.previous = tk.Button(self.window, text="←", command=lambda:Click(self.labelImage,self.labelPrice,self.labelFamily,0))
        self.next = tk.Button(self.window, text="→", command=lambda:Click(self.labelImage,self.labelPrice,self.labelFamily,1))
        self.searchById = tk.Button(self.window, text="Search", command=lambda:SearchById(self.inputId, self.labelImage, self.labelPrice, self.LabelFamily))
        self.inputId.pack()
        self.labelImage.pack()
        self.labelPrice.pack()
        self.labelFamily.pack()
        self.next.pack(side=tk.RIGHT)
        self.previous.pack(side=tk.LEFT)
        self.searchById.pack()
        self.window.mainloop()

Create tkinter windows

Deploy App

In [292]:
gui = GUI()
